In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
#torch.set_num_threads(1)

### 1 Learning to predict the Lorenz system using RNNs

In [ ]:
class CustomDataset(Dataset):
    """Sample random subsequences of length T_seq from the provided dataset.
    The dataset is a torch tensor of shape (T, N)."""
    def __init__(self, data, T_seq):
        # T x N
        self.data = data
        self.T_seq = T_seq

    def __getitem__(self, t):
        # t is the index of the first time step
        # return a sequence of length T_seq
        # and the sequence shifted by one time step
        return self.data[t:t+self.T_seq, :], self.data[t+1:t+self.T_seq+1, :]	

    def __len__(self):
        # sets the allowed range of t
        return len(self.data) - self.T_seq - 1


class BatchSampler():
    """Samples sequences from the dataset and stacks them into batches."""
    def __init__(self, dataset, batch_size):
        self.B = batch_size
        self.dataset = dataset

    def __call__(self):
        # get indices
        batch = [self.dataset[i] for i in self.get_random_inital_conditions()]

        # stack the sequences into separate batches
        xs = torch.stack([x for x, _ in batch])
        ys = torch.stack([y for _, y in batch])

        # reshape to (T, B, N)
        return xs.permute(1, 0, 2), ys.permute(1, 0, 2)
        
    def get_random_inital_conditions(self):
        # return a list of initial conditions of size self.B
        return torch.randperm(len(self.dataset))[:self.B]

In [ ]:
def train_RNN(
    rnn,
    output_layer,
    dataloader, 
    n_epochs, 
    print_every,
    lr=5e-4
):  
    # gather parameters
    rnn_params = list(rnn.parameters())
    output_layer_params = list(output_layer.parameters())

    # the optimizer performing stochastic gradient descent
    optimizer = torch.optim.Adam(rnn_params + output_layer_params, lr=lr)

    # the loss function
    criterion = nn.MSELoss()

    losses = []
    for epoch in range(n_epochs + 1):
        # get the data
        xs, ys = dataloader()

        # zero the gradients
        optimizer.zero_grad()

        # forward pass of the entire batch
        # implicitly initializes the hidden state
        # to zero!
        out, h = rnn(xs)
        y_pred = output_layer(out)

        # compute the loss
        loss = criterion(y_pred, ys)

        # backward pass, computes gradients
        loss.backward()

        # update the parameters
        optimizer.step()

        # store the loss
        losses.append(loss.item())

        # print the loss
        if epoch % print_every == 0:
            print('Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

    return losses

In [ ]:
# set the parameters
T_seq = ... # your code here
B = ... # your code here
epochs = ... # your code here
learning_rate = 5e-4 # you can play around with this setting

# load the data
X = torch.load('lorenz_data.pt')
print(X.size())

# initialize the dataset
dataset = CustomDataset(X, T_seq)

# initialize the dataloader
dataloader = BatchSampler(dataset, B)
xs, ys = dataloader()
print(xs.size(), ys.size())

# initialize for yourself!
rnn = ... # your code here
output_layer = ... # your code here

# train the model
losses = train_RNN(rnn, output_layer, dataloader, n_epochs=epochs, print_every=int(epochs/10), lr=learning_rate)